The following jupyter notebook demonstrates how a point file containing elk locations can be explored using an interactive map.

In [1]:
import geopandas as gpd # to work with geospatial data -- https://geopandas.org/en/stable/docs

In [2]:
# Load the data
location_file="data/ElkWinter08_09.csv"
location_df=gpd.read_file(location_file)

In [3]:
# Assign the coordinate reference system
location_df.crs = 'epsg:4326'

In [4]:
# View the data
location_df

,OBJECTID,CollarID,Date_MMDDY,Speed_mph,Heading,y,x,geometry
0,146467,843COW,3/5/2009,9.2,280,40.367308,-105.664583,None
1,146378,843COW,11/16/2008,1.2,345,40.358716,-105.656541,None
2,146816,856,11/7/2008,0,245,40.415933,-105.646525,None
3,68067,881,11/7/2008,0,215,40.410008,-105.644883,None
4,146812,856,11/7/2008,0,305,40.412341,-105.644525,None
...,...,...,...,...,...,...,...,...
10480,146823,856,11/8/2008,0,65,40.388416,-105.522908,None
10481,146821,856,11/8/2008,2.3,220,40.388241,-105.5224,None
10482,146822,856,11/8/2008,0,155,40.387675,-105.521108,None
10483,68072,881,11/8/2008,0,315,40.384808,-105.518541,None


In [5]:
# Create geometry values
location_df = gpd.GeoDataFrame(
    location_df, geometry=gpd.points_from_xy(location_df.x, location_df.y)
)

In [6]:
# View the data again
location_df

,OBJECTID,CollarID,Date_MMDDY,Speed_mph,Heading,y,x,geometry
0,146467,843COW,3/5/2009,9.2,280,40.367308,-105.664583,POINT (-105.66458 40.36731)
1,146378,843COW,11/16/2008,1.2,345,40.358716,-105.656541,POINT (-105.65654 40.35872)
2,146816,856,11/7/2008,0,245,40.415933,-105.646525,POINT (-105.64652 40.41593)
3,68067,881,11/7/2008,0,215,40.410008,-105.644883,POINT (-105.64488 40.41001)
4,146812,856,11/7/2008,0,305,40.412341,-105.644525,POINT (-105.64453 40.41234)
...,...,...,...,...,...,...,...,...
10480,146823,856,11/8/2008,0,65,40.388416,-105.522908,POINT (-105.52291 40.38842)
10481,146821,856,11/8/2008,2.3,220,40.388241,-105.5224,POINT (-105.52240 40.38824)
10482,146822,856,11/8/2008,0,155,40.387675,-105.521108,POINT (-105.52111 40.38768)
10483,68072,881,11/8/2008,0,315,40.384808,-105.518541,POINT (-105.51854 40.38481)


In [7]:
import folium # to make interactive maps
from folium.plugins import MarkerCluster, Draw # to group points on map and draw shapes

# Create map and show the data
# Get Rocky Mountain National Park coordinates - from https://www.google.com/maps
m = folium.Map(location=[40.384733209948294,-105.68988068298488,], zoom_start=5)

# Create and add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Create a list of location coordinates
geo_df_list = [[location.xy[1][0], location.xy[0][0]] for location in location_df.geometry]

# Create a marker for each location and add them to the cluster
for location in geo_df_list:
    marker = folium.Marker(location=location)
    marker_cluster.add_child(marker)

# Create and add a drawing control to the map
draw = Draw(export=True)
draw.add_to(m)

# show the map
m

# Beacuse there appears to be a lot of test data, draw a rectangle to be used as an exclusion zone
# Click the rectangle and copy the geojson into the next code cell 

In [8]:
# Load in shaply shape and polygon packages to convert our geojson to a polygon
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

# Store the geojson from earlier and convert to a polygon
polygon_geojson={"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-105.568657,40.357664],[-105.568657,40.369436],[-105.551834,40.369436],[-105.551834,40.357664],[-105.568657,40.357664]]]}}
polygon: Polygon = shape(polygon_geojson["geometry"])

# Exclude the data from within the polygon, storing it as a new filtered geodataframe
filtered_locations = location_df[~location_df.within(polygon)]

# Compare the number of locations before and after the filter
print(len(location_df))
len(filtered_locations)

10485


194

In [9]:
# Create a map of the filtered locations

m = folium.Map(location=[40.384733209948294,-105.68988068298488,], zoom_start=11)

filtered_locations.explore(m=m, marker_type="marker")

m

In [10]:
# Export the data
filtered_locations.to_csv("data/filtered_elk_locations.csv")

## Load in other data

In [11]:
## Get the elevation data
'''
go to https://planetarycomputer.microsoft.com/catalog
select "DEM" from the side nav and select "Copernicus DEM GLO-30"
copy the STAC Collection API URL e.g https://planetarycomputer.microsoft.com/api/stac/v1/collections/cop-dem-glo-30
and remove everything after "/collections/cop-dem-glo-30"

Note this standard uses Cloud Optimized GeoTiffs (COGS) 
which allow server-side clipping and overview images to speed
'''

api_url="https://planetarycomputer.microsoft.com/api/stac/v1"

from pystac_client import Client
import planetary_computer

client = Client.open(api_url,modifier=planetary_computer.sign_inplace)

In [12]:
# get elevation data continued
'''
Take the collection name from the API URL

'''
collection = 'cop-dem-glo-30'

# Access serach collection fror data around a point
# Load the shapely Point class to create a Point object
from shapely.geometry import Point

# Get Rocky Mountain National Park coordinates - from https://www.google.com/maps
# right click location, copy lat(y) and lng(x), and flip to conform to x,y
point = Point(-105.68988068298488, 40.384733209948294)  

# search the catalog
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)
print(len(search.item_collection()))

1


In [13]:
# get the first (and only) asset from the search
items=search.item_collection()
signed_asset = planetary_computer.sign(items[0].assets["data"])

In [14]:
# Create a tile layer since most browsers don't show Tiff files

from localtileserver import TileClient, get_folium_tile_layer # to visualize the geotif 
elevation_file=signed_asset.href
elevation_tiles = TileClient(elevation_file) # create tiles client
elevation_layer = get_folium_tile_layer(elevation_tiles, name='elevation') # create elevation tile layer


In [40]:
# create a map with the elevation data and elk locations
m = folium.Map([point.y,point.x], zoom_start = 10)
elevation_layer.add_to(m)

filtered_locations.explore(m=m, marker_type="marker")

m

In [38]:
# add elevation information to each row

#create raster layer for sampling
src = rasterio.open(elevation_file)

# create a list of coordinates for sampling
coord_list = [(x, y) for x, y in zip(filtered_locations['geometry'].x, filtered_locations['geometry'].y)]
# # save the samples back to the dataframe, and choose the first value
filtered_locations["elevation"] = [x[0] for x in src.sample(coord_list)]
filtered_locations.head()

,OBJECTID,CollarID,Date_MMDDY,Speed_mph,Heading,y,x,geometry,elevation
0,146467,843COW,3/5/2009,9.2,280,40.367308,-105.664583,POINT (-105.66458 40.36731),3315.504150
1,146378,843COW,11/16/2008,1.2,345,40.358716,-105.656541,POINT (-105.65654 40.35872),2984.461426
2,146816,856,11/7/2008,0,245,40.415933,-105.646525,POINT (-105.64652 40.41593),2801.393799
3,68067,881,11/7/2008,0,215,40.410008,-105.644883,POINT (-105.64488 40.41001),2606.151855
4,146812,856,11/7/2008,0,305,40.412341,-105.644525,POINT (-105.64453 40.41234),2633.158691


In [44]:
# Load in the park boundary data (Format GeoJSON)
import json
park_file="data/Rocky_Mountain_National_Park_-_Boundary_Polygon.geojson"
with open(park_file, 'r') as f:
    park_data = json.load(f)

In [46]:
# create a map with the boundary and elk locations
m = folium.Map([point.y,point.x], zoom_start = 10)

filtered_locations.explore(m=m, marker_type="marker")

folium.GeoJson(park_data,
    style_function=lambda feature:
        { 'color': 'red','opacity': 1, 'dashArray': '9', 'fillOpacity': 0.2, 'weight': 2},
        name="RMNP park boundary"
).add_to(m)

m

In [52]:
# Check if locations are in the park polygon

park_polygon: Polygon = shape(park_data["features"][0]["geometry"])

filtered_locations["within_park"] = ""
for index, row in filtered_locations.iterrows():
    print(row['geometry'])
    marker=Point(row['geometry'].x,row['geometry'].y)
    within=marker.within(park_polygon)
    # row['within_park']=within
    filtered_locations.loc[index, 'within_park'] = within

POINT (-105.664583 40.367308)
POINT (-105.656541 40.358716)
POINT (-105.646525 40.415933)
POINT (-105.644883 40.410008)
POINT (-105.644525 40.412341)
POINT (-105.644233 40.411333)
POINT (-105.644183 40.411191)
POINT (-105.644 40.41115)
POINT (-105.643533 40.410891)
POINT (-105.643416 40.4123)
POINT (-105.642575 40.415616)
POINT (-105.641991 40.414458)
POINT (-105.641483 40.373875)
POINT (-105.640833 40.358466)
POINT (-105.640383 40.3479)
POINT (-105.640366 40.410191)
POINT (-105.639216 40.409691)
POINT (-105.631858 40.366358)
POINT (-105.6249 40.347033)
POINT (-105.624716 40.408275)
POINT (-105.622133 40.355808)
POINT (-105.620341 40.359483)
POINT (-105.62015 40.332333)
POINT (-105.616908 40.355308)
POINT (-105.615308 40.348416)
POINT (-105.615108 40.360791)
POINT (-105.613833 40.335133)
POINT (-105.613325 40.338683)
POINT (-105.612866 40.375875)
POINT (-105.611075 40.357716)
POINT (-105.610291 40.373041)
POINT (-105.610016 40.347141)
POINT (-105.609966 40.3535)
POINT (-105.609191 40.3

AttributeError: 'NoneType' object has no attribute 'x'